In [1]:
def custom_train_step(model, inputs, targets, l2_strength):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = tf.keras.losses.binary_crossentropy(targets, predictions)

        # Calculate L2 regularization term for LSTM kernel and recurrent weights
        if l2_strength > 0.0:
            lstm_weights = model.layers[1].trainable_weights  # Access the LSTM layer's weights
            l2_reg = tf.add_n([tf.nn.l2_loss(w) for i,w in enumerate(lstm_weights) if i==1])
            loss += l2_strength * l2_reg

    # Compute gradients
    trainable_weights = model.trainable_variables
    gradients = tape.gradient(loss, trainable_weights)



    # non update kernel and bias
    for i, weight in enumerate(trainable_weights):
        if i==2:
            gradients[i] = tf.zeros_like(weight)


    opt.apply_gradients(zip(gradients, trainable_weights))
    return loss

def custom_train_loop(model, x_train, y_train, x_test, y_test, epochs, batch_size,l2_strength):
    num_batches = len(x_train) // batch_size
    
    #while epoch in range(100,200), since optimal mostly happen around this regime, we print middle epoch just on this conditon.
    for epoch in range(0,100):
        epoch_loss = 0.0
        for batch in range(num_batches):
            batch_inputs = x_train[batch * batch_size : (batch + 1) * batch_size]
            batch_targets = y_train[batch * batch_size : (batch + 1) * batch_size]

            # Reshape batch_targets to (batch_size, 1) to match the model output shape
            batch_targets = batch_targets.reshape(-1, 1)

            loss= custom_train_step(model, batch_inputs, batch_targets,l2_strength)
            epoch_loss += tf.reduce_mean(loss)

        avg_loss = epoch_loss / num_batches
        print("Epoch {}: Loss: {:.4f}".format(epoch + 1, avg_loss.numpy()))
        y_test_reshaped = y_test.reshape(-1, 1)
        val_loss,val_acc = model.evaluate(x_test, y_test_reshaped, verbose=0)

        print("Validation Accuracy: {:.2f}%".format(val_acc * 100))
        print("Validation Loss: {:.4f}".format(val_loss))
        
        path='unitestdense'+'/'+'weight.'+str(epoch+1)+'.txt'
        np.savetxt(path,model.layers[1].get_weights()[0])
        path1='unitestdense'+'/'+'weight_re.'+str(epoch+1)+'.txt'
        np.savetxt(path1,model.layers[1].get_weights()[1])
        path2='unitestdense'+'/'+'loss.'+str(epoch+1)+'.txt'
        np.savetxt(path2,[avg_loss.numpy(),val_loss,val_acc])
        path3='unitestdense'+'/'+'deweights.'+str(epoch+1)+'.txt'
        np.savetxt(path3,model.layers[1].get_weights()[2])
        path4='unitestdense'+'/'+'denseweights.'+str(epoch+1)+'.txt'
        np.savetxt(path4,model.layers[2].get_weights()[0])
        path5='unitestdense'+'/'+'densebweights.'+str(epoch+1)+'.txt'
        np.savetxt(path5,model.layers[2].get_weights()[1])
        
        
    #middle epochs, here now the batch_size is 128, so the mun_batches is 273, we devide it into 21 parts[13,26......273]    
    for epoch in range(100,201):
        div_batch=[13*i for i in range(1,20)]
        div_batch.append(num_batches)
        for i in range(len(div_batch)):
            for batch in range(div_batch[i]):
                epoch_loss = 0.0
                batch_inputs = x_train[batch * batch_size : (batch + 1) * batch_size]
                batch_targets = y_train[batch * batch_size : (batch + 1) * batch_size]

                # Reshape batch_targets to (batch_size, 1) to match the model output shape
                batch_targets = batch_targets.reshape(-1, 1)

                loss= custom_train_step(model, batch_inputs, batch_targets,l2_strength)
                epoch_loss += tf.reduce_mean(loss)

            avg_loss = epoch_loss / div_batch[i]
            print("Epoch {}: Loss: {:.4f}".format(epoch + div_batch[i]/num_batches, avg_loss.numpy()))


            # Reshape y_test to (batch_size, 1) to match the model output shape
            y_test_reshaped = y_test.reshape(-1, 1)
            val_loss,val_acc = model.evaluate(x_test, y_test_reshaped, verbose=0)

            print("Validation Accuracy: {:.2f}%".format(val_acc * 100))
            print("Validation Loss: {:.4f}".format(val_loss))
            
            path='unitestdense'+'/'+'weight.'+str(epoch+div_batch[i]/num_batches)+'.txt'
            np.savetxt(path,model.layers[1].get_weights()[0])
            path1='unitestdense'+'/'+'weight_re.'+str(epoch+div_batch[i]/num_batches)+'.txt'
            np.savetxt(path1,model.layers[1].get_weights()[1])
            path2='unitestdense'+'/'+'loss.'+str(epoch+div_batch[i]/num_batches)+'.txt'
            np.savetxt(path2,[avg_loss.numpy(),val_loss,val_acc])
            path3='unitestdense'+'/'+'deweights.'+str(epoch+div_batch[i]/num_batches)+'.txt'
            np.savetxt(path3,model.layers[1].get_weights()[2])
            path4='unitestdense'+'/'+'denseweights.'+str(epoch+div_batch[i]/num_batches)+'.txt'
            np.savetxt(path4,model.layers[2].get_weights()[0])
            path5='unitestdense'+'/'+'densebweights.'+str(epoch+div_batch[i]/num_batches)+'.txt'
            np.savetxt(path5,model.layers[2].get_weights()[1])
        
        
        
    for epoch in range(201,epochs):
        epoch_loss = 0.0
        for batch in range(num_batches):
            batch_inputs = x_train[batch * batch_size : (batch + 1) * batch_size]
            batch_targets = y_train[batch * batch_size : (batch + 1) * batch_size]

            # Reshape batch_targets to (batch_size, 1) to match the model output shape
            batch_targets = batch_targets.reshape(-1, 1)

            loss= custom_train_step(model, batch_inputs, batch_targets,l2_strength)
            epoch_loss += tf.reduce_mean(loss)

        avg_loss = epoch_loss / num_batches
        print("Epoch {}: Loss: {:.4f}".format(epoch + 1, avg_loss.numpy()))
        y_test_reshaped = y_test.reshape(-1, 1)
        val_loss,val_acc = model.evaluate(x_test, y_test_reshaped, verbose=0)

        print("Validation Accuracy: {:.2f}%".format(val_acc * 100))
        print("Validation Loss: {:.4f}".format(val_loss))
        
        path='unitestdense'+'/'+'weight.'+str(epoch+1)+'.txt'
        np.savetxt(path,model.layers[1].get_weights()[0])
        path1='unitestdense'+'/'+'weight_re.'+str(epoch+1)+'.txt'
        np.savetxt(path1,model.layers[1].get_weights()[1])
        path2='unitestdense'+'/'+'loss.'+str(epoch+1)+'.txt'
        np.savetxt(path2,[avg_loss.numpy(),val_loss,val_acc])
        path3='unitestdense'+'/'+'deweights.'+str(epoch+1)+'.txt'
        np.savetxt(path3,model.layers[1].get_weights()[2])
        path4='unitestdense'+'/'+'denseweights.'+str(epoch+1)+'.txt'
        np.savetxt(path4,model.layers[2].get_weights()[0])
        path5='unitestdense'+'/'+'densebweights.'+str(epoch+1)+'.txt'
        np.savetxt(path5,model.layers[2].get_weights()[1])
    
    

In [5]:
13/273

0.047619047619047616

In [ ]:
iiiiii=[]
for epoch in range(0,100) or in range(201,epochs):
    iiiiii.append(epoch)